In [1]:
# Step 1: Install Required Libraries
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install openai-whisper ipywidgets transformers pandas numpy scikit-learn fpdf youtube-transcript-api

Looking in indexes: https://download.pytorch.org/whl/cu118


In [45]:
# Step 2: Import Libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
from fpdf import FPDF
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
import re
import torch

# Step 3: Create Widgets for User Input
youtube_link = widgets.Text(
    placeholder="Enter YouTube URL",
    description="YouTube URL:",
    disabled=False
)

save_pdf = widgets.RadioButtons(
    options=["Yes", "No"],
    description="Download Transcript?",
    disabled=False
)

start_button = widgets.Button(description="Start Analysis")
output = widgets.Output()

# Step 4: Define Functions
def get_youtube_subtitles(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        text = " ".join([entry['text'] for entry in transcript])
        return text
    except Exception as e:
        print(f"❌ Error fetching subtitles: {e}")
        return None

def clean_text(text):
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()

def save_transcript_as_pdf(transcript, filename="transcript.pdf"):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, transcript)
        pdf.output(filename)
        print(f"✅ Transcript saved as {filename}")
    except Exception as e:
        print(f"❌ Error saving PDF: {e}")

def load_sentiment_model():
    try:
        model_name = "cardiffnlp/twitter-roberta-base-sentiment"
        sentiment_analyzer = pipeline("sentiment-analysis", model=model_name, device=0 if torch.cuda.is_available() else -1)
        print(f"✅ Loaded sentiment model: {model_name}")
        return sentiment_analyzer
    except Exception as e:
        print(f"⚠️ Error loading {model_name}, using default sentiment model.")
        return pipeline("sentiment-analysis")  # Fallback model

def analyze_sentiment(text, youtube_link):
    print(f"🔍 Performing sentiment analysis for: {youtube_link}")

    try:
        # Load sentiment model
        sentiment_analyzer = load_sentiment_model()

        # Define correct label mapping
        label_mapping = {
            "LABEL_0": "NEGATIVE",
            "LABEL_1": "NEUTRAL",
            "LABEL_2": "POSITIVE"
        }

        max_length = 512
        chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]

        results = []
        for chunk in chunks:
            result = sentiment_analyzer(chunk)
            for res in result:
                res['label'] = label_mapping.get(res['label'], "UNKNOWN")
            results.extend(result)

        # Calculate sentiment scores
        positive_score = sum(1 for r in results if r['label'] == 'POSITIVE')
        negative_score = sum(1 for r in results if r['label'] == 'NEGATIVE')
        neutral_score = sum(1 for r in results if r['label'] == 'NEUTRAL')

        overall_sentiment = max(
            [("POSITIVE", positive_score), ("NEGATIVE", negative_score), ("NEUTRAL", neutral_score)],
            key=lambda x: x[1]
        )[0]

        print("\n=== Sentiment Analysis ===")
        print(f"Overall Sentiment: {overall_sentiment}")
        print(f"Positive Score: {positive_score}")
        print(f"Negative Score: {negative_score}")
        print(f"Neutral Score: {neutral_score}")

        return {
            "Overall Sentiment": overall_sentiment,
            "Positive Score": positive_score,
            "Negative Score": negative_score,
            "Neutral Score": neutral_score,
            "Detailed Results": results
        }
    except Exception as e:
        print(f"❌ Error during sentiment analysis: {e}")
        return None

def summarize_text(text, youtube_link):
    print(f"📝 Performing summarization for: {youtube_link}")
    try:
        # Load a more powerful summarization model
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

        # Define the maximum input length for the summarizer
        max_input_length = 1024  # Model's maximum input length
        max_summary_length = 150  # Maximum length of the final summary
        min_summary_length = 50   # Minimum length of the final summary

        # Split the text into chunks if it's too long
        chunks = [text[i:i + max_input_length] for i in range(0, len(text), max_input_length)]

        summaries = []
        for chunk in chunks:
            # Generate summary for the chunk
            summary = summarizer(chunk, max_length=max_summary_length, min_length=min_summary_length, do_sample=False)
            summaries.append(summary[0]['summary_text'])

        # Combine all summaries into a final summary
        combined_summary = " ".join(summaries)

        # Summarize the combined summaries again to make it more concise
        final_summary = summarizer(
            combined_summary,
            max_length=max_summary_length,
            min_length=min_summary_length,
            do_sample=False
        )
        final_summary_text = final_summary[0]['summary_text']

        # Post-process the summary to remove specific names and details
        final_summary_text = re.sub(r'\b\w+ heel\b', 'the person', final_summary_text, flags=re.IGNORECASE)
        final_summary_text = re.sub(r'\b\w+ says\b', 'they mention', final_summary_text, flags=re.IGNORECASE)
        final_summary_text = re.sub(r'\b\w+ is\b', 'they are', final_summary_text, flags=re.IGNORECASE)

        print("\n✅ Final Summary Generated")
        return final_summary_text
    except Exception as e:
        print(f"❌ Error during summarization: {e}")
        return None

# Step 5: Handle Button Click
def on_start_button_click(b):
    with output:
        clear_output()
        print("⏳ Processing... Please wait.")

        try:
            video_id = youtube_link.value.split("v=")[1].split("&")[0]
        except Exception as e:
            print(f"❌ Invalid YouTube URL: {e}")
            return

        transcript = get_youtube_subtitles(video_id)
        if not transcript:
            print("❌ Failed to fetch subtitles.")
            return

        transcript = clean_text(transcript)
        print(f"✅ Subtitles fetched successfully!\n")

        # Perform sentiment analysis
        sentiment_results = analyze_sentiment(transcript, youtube_link.value)
        if sentiment_results:
            print("\n=== Sentiment Analysis ===")
            print(f"Overall Sentiment: {sentiment_results['Overall Sentiment']}")
            print(f"Positive Score: {sentiment_results['Positive Score']}")
            print(f"Negative Score: {sentiment_results['Negative Score']}")
            print(f"Neutral Score: {sentiment_results['Neutral Score']}")

        # Perform summarization
        summary = summarize_text(transcript, youtube_link.value)
        if summary:
            print("\n=== Summary ===")
            print(summary)
        else:
            print("❌ Summarization failed.")

        # Save as PDF if requested
        if save_pdf.value == "Yes":
            save_transcript_as_pdf(transcript)
        else:
            print("\n📄 Transcript not saved as PDF.")

# Attach the function to the button
start_button.on_click(on_start_button_click)

# Step 6: Display Widgets
display(youtube_link, save_pdf, start_button, output)

Text(value='', description='YouTube URL:', placeholder='Enter YouTube URL')

RadioButtons(description='Download Transcript?', options=('Yes', 'No'), value='Yes')

Button(description='Start Analysis', style=ButtonStyle())

Output()

In [46]:
!pip install pyannote.audio

from pyannote.audio import Pipeline

def identify_speakers(audio_file):
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")
    diarization = pipeline(audio_file)
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        print(f"Speaker {speaker} spoke from {turn.start:.1f}s to {turn.end:.1f}s")

# Example usage
identify_speakers("podcast_audio.wav")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 42.8 MB/s eta 0:00:00


Could not download 'pyannote/speaker-diarization' pipeline.
It might be because the pipeline is private or gated so make
sure to authenticate. Visit https://hf.co/settings/tokens to
create your access token and retry with:

   >>> Pipeline.from_pretrained('pyannote/speaker-diarization',
   ...                          use_auth_token=YOUR_AUTH_TOKEN)

If this still does not work, it might be because the pipeline is gated:
visit https://hf.co/pyannote/speaker-diarization to accept the user conditions.


TypeError: 'NoneType' object is not callable